In [2]:
import pylab
import numpy as np
import pandas as pd
import seaborn as sns
import datetime

# Read Data

In [6]:
data = pd.read_csv('./wandb_monsterkong.csv')
display(data)

,Name,Created,State,Final/CL Score,Final/Average Final Performance,Final/Average Online Performance,Final/Runtime (seconds),Test/Average.Mean reward per step,Test/Average.Mean reward per episode,current_task,setting/steps_per_task,hparams/ewc/sample_size_fim,hparams.learning_rate,hparams.train_freq
0,dqn-incremental_rl-MetaMonsterKong-v0_8t,2021-03-21T03:18:02.000Z,finished,39.300199,63.019324,30.993361,35908.333067,0.330103,63.019324,7,100000,NaN,0.000039,1.0
1,dqn-incremental_rl-MetaMonsterKong-v0_8t,2021-03-20T19:33:36.000Z,finished,36.162861,60.923913,22.638262,36056.701099,0.283733,60.923913,7,100000,NaN,0.000039,1.0
2,dqn-incremental_rl-MetaMonsterKong-v0_8t,2021-03-21T00:54:29.000Z,finished,36.303576,58.500000,27.843291,27172.262506,0.307486,58.500000,7,100000,NaN,0.000041,1.0
3,dqn-incremental_rl-MetaMonsterKong-v0_8t,2021-03-22T16:30:50.000Z,finished,35.181793,56.930233,26.693206,35704.682767,0.307785,56.930233,7,100000,NaN,0.000039,1.0
4,dqn-incremental_rl-MetaMonsterKong-v0_8t,2021-03-23T09:10:33.000Z,finished,35.674397,55.896861,30.725013,35473.623165,0.315306,55.896861,7,100000,NaN,0.000039,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,dqn-incremental_rl-MetaMonsterKong-v0_8t,2021-03-19T06:50:43.000Z,finished,7.476939,1.500000,26.714148,34836.182098,0.003000,1.500000,7,100000,NaN,0.001828,1.0
302,dqn-incremental_rl-MetaMonsterKong-v0_8t,2021-03-18T06:11:09.000Z,finished,5.236737,1.500000,17.280983,14430.257273,0.003000,1.500000,7,200000,NaN,0.007080,10.0
303,dqn-incremental_rl-MetaMonsterKong-v0_8t,2021-03-18T23:47:04.000Z,finished,7.880196,1.125000,29.024797,32573.330557,0.002250,1.125000,7,100000,NaN,0.009225,1.0
304,ppo-incremental_rl-MetaMonsterKong-v0_8t,2021-03-18T06:17:56.000Z,finished,0.494856,0.500000,0.828125,36663.834999,0.001000,0.500000,7,200000,NaN,0.002984,NaN


# Plot data

In [ ]:
!rm ./*.pdf # cleanup existing pdfs in folder
from matplotlib.lines import Line2D # import necessary for the legend

pylab.close('all') # close all open figures

settings = np.unique(data['setting']) # get list of all possible settings datasets and methods
datasets = np.unique(data['dataset'])
methods = list(set(data['method']))

colors = sns.color_palette('hls', len(methods)) # assign a color to each method

data['Final/Runtime (seconds)'] /= data['Final/Runtime (seconds)'].std() # standarize bubble size

time = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M") # get current timestamp to save files

# loop over settings and datasets, each one is a plot
for setting in settings:
    for dataset in datasets:  
        d = data[(data['setting'] == setting) & (data['dataset'] == dataset)] # filter by settings and dataset
        # obtain values to plot
        x = d['Final/Average Online Performance'].values
        y = d['Final/Average Final Performance'].values
        z = d['Final/Runtime (seconds)'].values
        
        # manually remove cartpole and plots with less than 2 bubbles
        if len(x[x==x]) < 2 and len(y[y==y]) < 2 or 'CartPole' in data['dataset']:
            continue
            
        fig, ax = pylab.subplots() # create axis
        z = z * 1000 # make the bubbles bigger
        m = [methods.index(_m) for _m in d['method'].values] # map method string to integer id
        c = [colors[_m] for _m in m] # use the integer id to get the corresponding color
        
        ax.scatter(x, y, z, c, alpha=0.5) # plot

        # make legend and format axis
        legend_elements = [
             Line2D([0], [0], marker='o', color='w', label=_m,
                          markerfacecolor=colors[methods.index(_m)], markersize=15, alpha=0.5) for _m in set(d['method'].values)
        ]
        xoffset = np.abs(x.max() - x.min()) * 0.25
        yoffset = np.abs(y.max() - y.min()) * 0.25

        pylab.xlim([x.min() - xoffset, x.max() + xoffset])
        pylab.ylim([y.min() - yoffset, y.max() + yoffset])
        ax.legend(handles=legend_elements, loc='lower right')
        pylab.xlabel("Online Performance")
        pylab.ylabel("Final Performance")
        pylab.savefig(f"setting={setting}_dataset={dataset}_{time}.pdf")
        
print(time)

In [ ]:
!tar -czvf rl_plots_absolute_size.tar.gz ./*.pdf